In [1]:
from google.colab import files

uploaded_model1 = files.upload()





Saving model3a.csv to model3a.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score
from sklearn.decomposition import NMF
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model

# Load the dataset
data = pd.read_csv('model3a.csv')

# Preprocess the data
# Convert labels to one-hot encoding
labels = data.iloc[:, 1:]
text_data = data['text']

# Train-test split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Matrix Factorization (NMF)
optimal_num_topics = 5
nmf = NMF(n_components=optimal_num_topics)
W = nmf.fit_transform(labels)
H = nmf.components_

# Neural Collaborative Filtering (NCF)
num_users = labels.shape[0]
num_items = labels.shape[1]
embedding_dim = 64

# Define NCF model
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

user_embedding = Embedding(num_users, embedding_dim)(user_input)
item_embedding = Embedding(num_items, embedding_dim)(item_input)

user_flat = Flatten()(user_embedding)
item_flat = Flatten()(item_embedding)

concat = Concatenate()([user_flat, item_flat])

dense_1 = Dense(64, activation='relu')(concat)
dense_2 = Dense(32, activation='relu')(dense_1)

output = Dense(1, activation='sigmoid')(dense_2)

ncf_model = Model(inputs=[user_input, item_input], outputs=output)
ncf_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train NCF model
user_indices = train_data.index
item_indices = range(num_items)
labels_train = labels.values

# Define positive and negative samples for training
def generate_samples():
    for user in user_indices:
        for item in item_indices:
            yield user, item, labels_train[user, item]

samples = np.array(list(generate_samples())).T
users_train = samples[0]
items_train = samples[1]
labels_train = samples[2]

ncf_model.fit([users_train, items_train], labels_train, epochs=20, batch_size=16)

# Recommendation for a given text
def recommend_labels(text, threshold=0.5):
    # Predict labels using NCF model
    user_idx = np.array([0] * num_items)  # Assuming user index 0 for recommendation
    item_idx = np.array(range(num_items))
    scores = ncf_model.predict([user_idx, item_idx])

    # Apply thresholding to get binary predictions
    binary_predictions = (scores > threshold).astype(int)

    # Get the indices of recommended labels based on thresholding
    recommended_labels = np.where(binary_predictions == 1)[0]

    return recommended_labels

# Example: Recommend labels for a given text
text_user_input = "One of the proudest accomplishments of my life was earning my college degree, despite the fact that my early adulthood pointed in the opposite direction, beginning with my marriage at the age of 19. Throughout the 1990s I lived as one of the working poor,someone who slipped through the cracks of supposedly historic prosperity"
recommended_labels = recommend_labels(text_user_input)

# Print recommended labels
print("Recommended Microcredentials for the input text:")
for label in recommended_labels:
    print(f"Label {label}: {labels.columns[label]}")


Epoch 1/20
26372/26372 [==============================] - 175s 7ms/step - loss: 0.1169 - accuracy: 0.9652
Epoch 2/20
26372/26372 [==============================] - 168s 6ms/step - loss: 0.1052 - accuracy: 0.9676
Epoch 3/20
26372/26372 [==============================] - 168s 6ms/step - loss: 0.0953 - accuracy: 0.9709
Epoch 4/20
26372/26372 [==============================] - 163s 6ms/step - loss: 0.0894 - accuracy: 0.9724
Epoch 5/20
26372/26372 [==============================] - 164s 6ms/step - loss: 0.0854 - accuracy: 0.9733
Epoch 6/20
26372/26372 [==============================] - 164s 6ms/step - loss: 0.0817 - accuracy: 0.9745
Epoch 7/20
26372/26372 [==============================] - 164s 6ms/step - loss: 0.0784 - accuracy: 0.9758
Epoch 8/20
26372/26372 [==============================] - 161s 6ms/step - loss: 0.0754 - accuracy: 0.9769
Epoch 9/20
26372/26372 [==============================] - 160s 6ms/step - loss: 0.0724 - accuracy: 0.9778
Epoch 10/20
26372/26372 [=====================